## CMSC 35300 Final Project: Lasso Models
Shweta Kamath <br>
Nivedita Vatsa <br>
Carolyn Vilter

#### Setup

Source: https://www.kaggle.com/code/mcweng24/lasso-regression-using-numpy

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltb
import seaborn as sns

In [21]:
import scipy.io as sio

In [22]:
# Import data
df = pd.read_csv("data/all_data_standardized.csv")

In [23]:
# Separate out Xs
X = df.loc[:, ~df.columns.isin(["child_id", "mother_id", "treat_alike_scale", "treat_alike_binary"])]
X = X.to_numpy()

# add columns of 1s
X = np.hstack((np.ones((len(X), 1)), X))

# Separate out two prospective ys
y_scale = df.loc[:, df.columns == "treat_alike_scale"]
y_scale = y_scale.to_numpy()

y_binary = df.loc[:, df.columns == "treat_alike_binary"]
y_binary = y_binary.to_numpy()

In [32]:
# matlab_data_file = sio.loadmat('face_emotion_data.mat')
# X = matlab_data_file['X']
# y_scale = matlab_data_file['y']

In [45]:
# # load data, make sure ‘fisheriris.mat‘ is in your working directory
# data = sio.loadmat("fisheriris.mat")

# # training data
# X = data['meas']
# y_scale = data['species']

### Lasso Regression
Predict repeatedly using cross validation; plot test error.

In [25]:
def predicted_values(X, w):
    # X will be n x (d+1)
    # w will be (d+1) x 1
    predictions = np.dot(X, w) # n x 1
    return(predictions)


def rho_compute(y,X,w,j):
    #y is the response variable
    #X is the predictor variables matrix
    #w is the weight vector
    #j is the feature selector
    X_k = np.delete(X,j,1) #Remove the j variable i.e. j column
    w_k = np.delete(w,j) #Remove the weight j
    predict_k = predicted_values(X_k, w_k)
    residual = y - predict_k
    rho_j = np.sum(X[:,j]*residual)
    return(rho_j)


#z computation for unnormalised features
def z_compute(X):
    z_vector = np.sum(X*X, axis = 0) #Compute sum for each column
    return(z_vector)


def coordinate_descent(y,X,w,alpha,z,tolerance):
    max_step = 100.
    iteration = 0
    while(max_step > tolerance):
        iteration += 1
        #print("Iteration (start) : ",iteration)
        old_weights = np.copy(w)
        #print("\nOld Weights\n",old_weights)
        for j in range(len(w)): #Take the number of features ie columns
            rho_j = rho_compute(y,X,w,j)
            if j == 0: #Intercept is not included with the alpha regularisation
                w[j] = rho_j/z[j]
            elif rho_j < -alpha*len(y):
                w[j] = (rho_j + (alpha*len(y)))/z[j]
            elif rho_j > -alpha*len(y) and rho_j < alpha*len(y):
                w[j] = 0.
            elif rho_j > alpha*len(y):
                w[j] = (rho_j - (alpha*len(y)))/z[j]
            else:
                w[j] = np.NaN
        #print("\nNew Weights\n",w)
        step_sizes = abs(old_weights - w)
        #print("\nStep sizes\n",step_sizes)
        max_step = step_sizes.max()
        #print("\nMax step:",max_step)
        
        
    return(w, iteration, max_step)

In [46]:
w = np.zeros(X.shape[1], dtype = float) # initialize w
z = z_compute(X) # pre-compute the z_j term


# set the alpha and tolerance level
alpha = 0.0
tolerance = 0.0000001

In [50]:
w_opt, iterations, max_step = coordinate_descent(y_scale, X, w, alpha, z, tolerance)

UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [43]:
print(w_opt)
print(iterations)
print(max_step)

[-2.59990365e+295  2.04102411e+297 -2.49954287e+299  3.04779224e+301
 -1.52466460e+303  1.63173460e+305             -inf              nan
              nan]
19
nan
